In [2]:
import os
import sys

current_path = os.getcwd()
current_path = os.path.join(current_path, 'Dataset')
print(current_path)
folder_com = os.path.join(current_path, 'Commercial Aero')
folder_uss = os.path.join(current_path, 'USS')

def get_file_as_id(folder_path):
    if not os.path.exists(folder_path):
        return ValueError
    folder_name = folder_path.split('/')[-1]
    id = folder_name.split('.')[0]

    return id

for file in os.listdir(folder_uss):
    print(get_file_as_id(os.path.join(folder_uss, file)))
    

/home/user/AI-Agents-For-ST/dsRAG/Dataset
USS_Satcom_Gilat_2022
USS_Satcom_Comtech_2022
USS_Satcom_Comtech_2021
USS_Satcom_Viasat_2023
USS_Satcom_Gilat_2021
USS_Satcom_Echostar_2021
USS_Satcom_Viasat_2022
USS_Satcom_Echostar_2022


In [26]:
from dsrag.knowledge_base import KnowledgeBase
from dsrag.llm import OpenAIChatAPI
from dsrag.reranker import NoReranker

from dsrag.database.vector.chroma_db import ChromaDB
from dsrag.document_parsing import extract_text_from_pdf

llm = OpenAIChatAPI(model='gpt-4o-mini')

#create knowledge base
kb_dict = {}

def setup_kb(folder_path):   
    for file in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file)
        if file.endswith('.pdf'):
            id = get_file_as_id(file_path)
            kb_dict[id] = KnowledgeBase(id, reranker = NoReranker(), vector_db=ChromaDB(id), storage_directory="~/AI-Agents-For-ST/dsRAG/storage")
            text = extract_text_from_pdf(file_path)
            kb_dict[id].add_document(doc_id=id, text=text[0])
            
            
setup_kb(folder_uss)


Adding 990 chunks to the database
Adding 792 chunks to the database
Adding 777 chunks to the database
Adding 485 chunks to the database
Adding 941 chunks to the database
Adding 667 chunks to the database
Adding 504 chunks to the database
Adding 691 chunks to the database


In [ ]:
kb_dict = {}

def setup_kb(folder_path):   
    for file in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file)
        if file.endswith('.pdf'):
            id = get_file_as_id(file_path)
            kb_dict[id] = KnowledgeBase(id, reranker = NoReranker(), vector_db=ChromaDB(id), storage_directory="~/AI-Agents-For-ST/dsRAG/storage")
            
            
for folder in [folder_com, folder_uss]:
    setup_kb(folder)